In [ ]:
from __future__ import print_function
import os
import tensorflow as tf
import cv2
import numpy as np

DATASET_PATH = os.getcwd() + '/TrainData'

MODE = 'folder'

IMG_HEIGHT = 100
IMG_WIDTH = 100
CHANNELS = 3

N_CLASSES = 1

save_path = 'model'
model_name = 'CNN'

map_file_name = 'map_categorical_classes.txt'

if not os.path.exists(save_path):
    os.makedirs(save_path)

full_save_path = os.path.join(save_path, model_name)

def create_map_file(file_name, classes, class_to_label_dict):
    with open(file_name,'w') as map_file:
        for _class in classes:
            label = class_to_label_dict[_class]
            map_file.write(str(label)+" "+_class+" "+os.linesep)


def read_image(path):
    path = str(path, 'utf-8')
    return cv2.imread(path, cv2.IMREAD_COLOR)

def read_data(path, label):
    image = read_image(path)
    label = np.array(label, dtype=np.int32)
    image = image.astype(np.int32)
    return image, label
    
def data_resize_fuction(image_decoded, label):
    image_decoded.set_shape([None, None, CHANNELS])
    image_resized = tf.image.resize_images(image_decoded, [IMG_HEIGHT,IMG_WIDTH])
    return image_resized, label

def data_normalization(image, label):
    image = tf.image.per_image_standardization(image)
    return image, label

def input_data(dataset_path, mode, batch_size):
    imagepaths, labels, classes = list(), list(), list()
    class_to_label_dict = {}
    
    label= 0
    
    
    if mode == 'folder':
        files = os.listdir(DATASET_PATH)
        if '.DS_Store' in files:
            files.remove('.DS_Store')
        for file in files:
            _class = file.split('_')[0]
            if not _class in classes:
                classes.append(_class)
                class_to_label_dict[_class] = label
                label += 1
            if file.endswith('.jpg'):
                imagepaths.append(os.path.join(DATASET_PATH, file))
                labels.append(class_to_label_dict[_class])
        print(class_to_label_dict)
        create_map_file(map_file_name, classes, class_to_label_dict)
        
        global N_CLASSES
        N_CLASSES = len(classes)
        print(classes)
        
    else:
        raise Exception("Unknown mode")
    
    dataset = tf.data.Dataset.from_tensor_slices((imagepaths, labels))
    dataset = dataset.map(lambda images, labels:
                         tuple(tf.py_func(read_data, [images, labels], [tf.int32, tf.int32])))
    
    dataset = dataset.map(data_resize_fuction)
    dataset = dataset.map(data_normalization)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(buffer_size=(int(len(imagepaths) * 0.4) + 3 * batch_size))
    dataset = dataset.batch(batch_size)
 
    iterator = dataset.make_initializable_iterator()
    image_stacked, label_stacked = iterator.get_next()
    next_element = iterator.get_next()

    with tf.Session() as sess:
        sess.run(iterator.initializer)
        image, label = sess.run([image_stacked, label_stacked])

    return dataset
    

learning_rate = 0.001
num_steps = 100
batch_size = 128
display_step = 10
dropout = 0.75



dataset = input_data(DATASET_PATH, MODE, batch_size)

def conv_net(x, n_classes, dropout, reuse, is_training):
    with tf.variable_scope('ConvNet', reuse=reuse):
        ##change value later
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
        
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        
        #fully connected
        affine = tf.contrib.layers.flatten(conv2)
        affine = tf.layers.dense(affine,1024)
        affine = tf.layers.dropout(affine, rate=dropout, training = is_training)
        
        out = tf.layers.dense(affine, n_classes)
        out = tf.nn.softmax(out) if not is_training else out
    return out

iterator = dataset.make_initializable_iterator()
X, Y = iterator.get_next()

print("N_CLASSES")
print(N_CLASSES)
## false로 했을때 value error
print(X)
logits_train = conv_net(X, N_CLASSES, dropout, reuse=False, is_training=True)
logits_test = conv_net(X, N_CLASSES, dropout, reuse=True, is_training=False)

loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=logits_train, labels=Y))


optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.cast(Y, tf.int64))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(iterator.initializer)
    
    
    for step in range(1, num_steps+1):
        print(step)
        if step % display_step == 0:
            _, loss, acc = sess.run([train_op, loss_op, accuracy])
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        else:
            sess.run(train_op)
            
    print("Optimization Finished!")
    saver.save(sess, full_save_path)


In [ ]:
##test##

# import cv2
# import tensorflow as tf
# import os
# import numpy as np

CHANNELS = 3
IMG_HEIGHT = 100
IMG_WIDTH = 100
save_path = 'model'
model_name = 'CNN'
full_save_path = os.path.join(save_path, model_name)

meta_graph = full_save_path + '.meta'

map_file_name = "map_categorical_classes.txt"
mode = ['label_to_class', 'class_to_label']


image_folder_path = "/Users/NamHyunsil/Desktop/test/jenny/crop_image"

def create_map_dict(file_name, mode):
    map_dict = {}
    with open(file_name,'r') as map_file:
            lines = map_file.readlines()
            for line in lines:
                split = line.split(' ')
                label = int(split[0])
                _class = split[1]
                if mode == 'label_to_class':
                    map_dict[label] = _class
                elif mode == 'class_to_label':
                    map_dict[_class] = label
                else :
                    raise Exception("Unknown mode")
    return map_dict

label_to_class_dict = create_map_dict(map_file_name, mode[0])


def run_test(image_path):
    image_np = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image_np = image_np.astype(np.int32)
    image_tf = tf.convert_to_tensor(image_np, tf.int32)
    image_tf.set_shape([None, None, CHANNELS])
    image = tf.image.resize_images(image_tf, [IMG_HEIGHT,IMG_WIDTH])
    image = tf.image.per_image_standardization(image)
    image = tf.expand_dims(image,0)
    X = image

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.import_meta_graph(meta_graph)
        saver.restore(sess, full_save_path)
        logits_test = conv_net(X, N_CLASSES, dropout, reuse=True, is_training=False)
        print(sess.run(logits_test))
        result = tf.argmax(logits_test, 1)
        result = sess.run(result)[0]
        print(label_to_class_dict[result])

    
images = os.walk(image_folder_path).__next__()[2]
images.remove('.DS_Store')

for image in images:
    image_path = os.path.join(image_folder_path, image)
    print(image_path)
    run_test(image_path)
    print()

